In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time as tm
import sys
import pickle as pkl
import matplotlib.pyplot as plt
# from numpy import linalg as LA
import scipy.linalg as LA
import os
import yaml
from pathlib import Path

import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/model/mmnet')
sys.path.append(os.path.dirname(os.getcwd()))


from model.mmnet.utils import *
from data.sample_generator import *
from model.mmnet.MMNet import *


In [2]:
################################################################################
#
####                              SETTINGS
#
################################################################################

######################
###  Load parameters of the system ###
######################
dirPath = os.path.dirname(os.getcwd())
with open(dirPath + '/config.yml', 'r') as f:
    aux = yaml.load(f,  Loader=yaml.FullLoader)
config = dict2namespace(aux)


######################
###  General setup ###
######################
SEED = 123
torch.manual_seed(SEED)
useGPU = False # If true, and GPU is available, use it.
loadChannelInput = True

saveModel = False
dirPath = str(Path(os.getcwd()).parent.absolute())

useGPU = True # If true, and GPU is available, use it.

#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda:0'
else:
    device = 'cpu'

#Parameters
train_iter = 10000
train_batch_size = 500
learning_rate = 1e-3
num_layers = 10


if loadChannelInput == True:

    # \\ IID channels
    with open(dirPath + '/data/Hiid_5000bs_3264', 'rb') as fp:
        H = pkl.load(fp)
    batch_size = H.shape[0]

    #Create generator
    generator = sample_generator(batch_size, config.mod_n, config.NR)
else:
    batch_size = 5000
    generator = sample_generator(batch_size, config.mod_n, config.NR)
    H, y, x, j_indices, noise_sigma = generator.give_batch_data(config.NT, 
                                                                snr_db_min=config.SNR_dBs[config.NT][0], snr_db_max=config.SNR_dBs[config.NT][0], 
                                                                batch_size=batch_size, correlated_flag=config.corr_flag, rho=config.rho)  


def train(H_test, model, optimizer, generator, config, device='cuda'):
    criterion = nn.MSELoss().to(device=device)
    model.train()
    real_QAM_const = generator.real_QAM_const.to(device=device)
    imag_QAM_const = generator.imag_QAM_const.to(device=device)

    for i in range(train_iter):

        H, y, x, j_indices, noise_sigma = generator.give_batch_data(config.NT, 
                                                                    snr_db_min=config.SNR_dBs[config.NT][0], snr_db_max=config.SNR_dBs[config.NT][-1], 
                                                                    batch_size=train_batch_size)  
        
        H = H.to(device=device)
        y = y.to(device=device)
        noise_sigma = noise_sigma.to(device=device)
        
        list_batch_x_predicted = model.forward(H, y, noise_sigma, train_batch_size)

        x = x.to(device=device)
        j_indices = j_indices.to(device=device)

        loss, SER = loss_fn(x, list_batch_x_predicted, j_indices, real_QAM_const, imag_QAM_const, criterion, num_layers)                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        del y, x, j_indices, noise_sigma, list_batch_x_predicted

        if (i%500==0):
            model.eval()
            H_test = H_test.to(device='cpu')
            y, x, j_indices, noise_sigma = generator.give_batch_data_Hinput(H_test, config.NT, snr_db_min=config.SNR_dBs[config.NT][0], snr_db_max=config.SNR_dBs[config.NT][-1],
                                                                         batch_size=batch_size)            
            H_test = H_test.to(device=device)
            y = y.to(device=device)
            noise_sigma = noise_sigma.to(device=device)

            with torch.no_grad():
                list_batch_x_predicted = model.forward(H_test, y, noise_sigma, batch_size)
                x = x.to(device=device)
                j_indices = j_indices.to(device=device)
                loss_last, SER_final = loss_fn(x, list_batch_x_predicted, j_indices, real_QAM_const, imag_QAM_const, criterion, num_layers)                
                results = [loss_last.detach().item(), 1 - SER_final]
                print_string = [i]+results
                print(' '.join('%s' % np.round(x,6) for x in print_string))
            del y, x, j_indices, noise_sigma, list_batch_x_predicted
            model.train()

In [3]:

model = MMNet(num_layers, config.NT, config.NR, generator.constellation, device=device)
model = model.to(device=device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(H, model, optimizer, generator, config, device)

0 0.496112 0.979212
500 0.161388 0.605181
1000 0.093234 0.296638
1500 0.066246 0.182238
2000 0.052808 0.112563
2500 0.048206 0.082987
3000 0.045975 0.06345
3500 0.04523 0.058488
4000 0.045186 0.0561
4500 0.044541 0.054538
5000 0.043226 0.047225
5500 0.040987 0.038456
6000 0.040065 0.036312
6500 0.039167 0.03285
7000 0.038903 0.031863
7500 0.038408 0.028381
8000 0.038166 0.025706
8500 0.037784 0.024288
9000 0.037657 0.023294
9500 0.037781 0.0239


In [4]:
## generator = sample_generator(test_batch_size, mod_n, NR)
accs_NN = model_eval(config.NT, model, config.SNR_dBs[config.NT][0], config.SNR_dBs[config.NT][-1], batch_size, generator, device, num_layers, iterations=500,  test_set_flag = True, test_set = H)
accs_NN

serlist = []
for ii in range(len(config.SNR_dBs[config.NT])):
    serlist.append(accs_NN[ii][1])

with open(dirPath + '/results/mmnet_results', "wb") as output_file:
    pkl.dump(serlist, output_file)

/home/nicolas/MIMO_detection_project/Langevin_repo/Langevin-MIMO-detector/model/mmnet/utils.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  H = torch.tensor(test_set)
